## Data

In [1]:
import torch
import torch.nn as nn
import torchtext; torchtext.disable_torchtext_deprecation_warning()
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

corpus = [
    "i like cats and dogs",
    "books are expensive"    
]
data_size = len(corpus)

# 0: noun/pronoun - 1: verb - others - 2
labels = [[0, 1, 0, 2, 0],
          [0, 1, 2]]

# Define the max vocabulary size and sequence length
vocab_size = 10
sequence_length = 5

In [2]:
# Define tokenizer function
tokenizer = get_tokenizer('basic_english')

# Create a function to yield list of tokens
def yield_tokens(examples):
    for text in examples:
        yield tokenizer(text)

# Create vocabulary
vocab = build_vocab_from_iterator(yield_tokens(corpus),
                                  max_tokens=vocab_size,
                                  specials=["<unk>", "<pad>"])
vocab.set_default_index(vocab["<unk>"])
vocab.get_stoi()

{'dogs': 6,
 'books': 4,
 'like': 9,
 'i': 8,
 'cats': 5,
 'are': 3,
 'expensive': 7,
 'and': 2,
 '<pad>': 1,
 '<unk>': 0}

In [3]:
# Tokenize and numericalize your samples
def vectorize(text, vocab, sequence_length, sequence_label):
    tokens = tokenizer(text)
    
    token_ids = [vocab[token] for token in tokens]    
    token_ids = token_ids + [vocab["<pad>"]] * (sequence_length - len(tokens))
    sequence_label = sequence_label + [3] * (sequence_length - len(tokens))
    
    return torch.tensor(token_ids, dtype=torch.long), torch.tensor(sequence_label, dtype=torch.long)

# Vectorize the samples
sentence_vecs = []
label_vecs = []
for sentence, labels in zip(corpus, labels):
    sentence_vec, labels_vec = vectorize(sentence, vocab, sequence_length, labels)
    sentence_vecs.append(sentence_vec)
    label_vecs.append(labels_vec)

In [4]:
for v in sentence_vecs:
    print(v)

tensor([8, 9, 5, 2, 6])
tensor([4, 3, 7, 1, 1])


In [5]:
for v in label_vecs:
    print(v)

tensor([0, 1, 0, 2, 0])
tensor([0, 1, 2, 3, 3])


## Model

In [6]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, ff_dim):
        super().__init__()
        self.attn = nn.MultiheadAttention(embed_dim=embed_dim, 
                                          num_heads=num_heads)
        self.ffn = nn.Linear(in_features=embed_dim, 
                             out_features=ff_dim)
        self.layernorm_1 = nn.LayerNorm(normalized_shape=embed_dim)
        self.layernorm_2 = nn.LayerNorm(normalized_shape=embed_dim)

    def forward(self, query, key, value):
        attn_output, _ = self.attn(query, key, value)
        out_1 = self.layernorm_1(query + attn_output)
        ffn_output = self.ffn(out_1)
        x = self.layernorm_2(out_1 + ffn_output)
        return x

In [7]:
class POS_Model(nn.Module):
    def __init__(self, vocab_size, num_classes):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, 5)
        self.transformer = TransformerBlock(5, 1, 5)
        self.fc = nn.Linear(5, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        x = self.transformer(x, x, x)
        x = self.fc(x)
        return x.permute(0, 2, 1)

num_classes = 4
model = POS_Model(vocab_size, num_classes)

# test
data = torch.tensor([[3, 2, 3, 2, 1]]).long()
output = model(data)
print(output.shape)

torch.Size([1, 4, 5])


## Train with full data

In [8]:
criterion = nn.CrossEntropyLoss(ignore_index=3)
optimizer = torch.optim.Adam(model.parameters(), lr=0.2)

In [9]:
input_data = torch.tensor( [[8, 9, 5, 2, 6],
                            [4, 3, 7, 1, 1]], dtype=torch.long)
label_data = torch.tensor([[0, 1, 0, 2, 0],
                           [0, 1, 2, 3, 3]], dtype=torch.long)

for _ in range(30):
    optimizer.zero_grad()
    outputs = model(input_data)
    loss = criterion(outputs, label_data)
    print(loss.item())
    loss.backward()
    optimizer.step()

1.635465145111084
1.143448829650879
0.7312991619110107
0.5712199211120605
0.46315231919288635
0.40208670496940613
0.30497029423713684
0.3893255293369293
0.2814027667045593
0.3158930838108063
0.27431923151016235
0.263588547706604
0.26233571767807007
0.2334529161453247
0.2404129058122635
0.24104616045951843
0.22688882052898407
0.22978930175304413
0.22981111705303192
0.2158343493938446
0.21523970365524292
0.19061993062496185
0.8124179244041443
0.08971190452575684
1.3433516025543213
0.14286266267299652
0.13118648529052734
0.15590038895606995
0.18190790712833405
0.1534910947084427


In [10]:
outputs = model(input_data)
o_softmax = torch.softmax(outputs, axis=1)

print(o_softmax[0, :, 0].detach().numpy())
print(o_softmax[0, :, 1].detach().numpy())
print(o_softmax[0, :, 2].detach().numpy())
print(o_softmax[0, :, 3].detach().numpy())
print(o_softmax[0, :, 4].detach().numpy())

print()
print(o_softmax[1, :, 0].detach().numpy())
print(o_softmax[1, :, 1].detach().numpy())
print(o_softmax[1, :, 2].detach().numpy())
print(o_softmax[1, :, 3].detach().numpy())
print(o_softmax[1, :, 4].detach().numpy())

[9.941712e-01 8.765651e-05 5.702931e-03 3.814187e-05]
[5.8304160e-03 9.9416935e-01 3.5292735e-10 2.4517695e-07]
[0.57475835 0.15128829 0.2723962  0.0015572 ]
[1.51460115e-02 1.45601434e-05 9.84801233e-01 3.81421742e-05]
[9.9769968e-01 2.0392374e-03 2.2697393e-04 3.4082910e-05]

[9.8629469e-01 3.5072331e-05 1.3628336e-02 4.2032367e-05]
[5.5610351e-03 9.9443877e-01 3.2491745e-10 2.3713858e-07]
[2.5966820e-01 1.1142627e-02 7.2849166e-01 6.9749792e-04]
[2.1842964e-01 6.3310033e-08 7.8155071e-01 1.9550060e-05]
[2.1860878e-01 6.3409104e-08 7.8137153e-01 1.9563035e-05]


In [11]:
# [[0, 1, 0, 2, 0],
#  [0, 1, 2, *, *]]

o_softmax.argmax(axis=1)

tensor([[0, 1, 0, 2, 0],
        [0, 1, 2, 2, 2]])